R 🪨 Rocas = obstáculo o terreno costoso.

A 🏜️ Arena fina = coste alto de movimiento (más energía).

ZR ⛰️ Zona rocosa = no transitable.
ZA   Zanjas =no transitable
C 📡 Zona de comunicación = debe volver aquí a enviar datos.

PI 🧪 Puntos de interés = debe visitar para recolectar muestras.

Rover = Rover autonomo

| Símbolo     | Tipo de terreno         | Coste de movimiento | Descripción                             |
| -------     | ----------------------- | ------------------- | --------------------------------------- |
| `.`         | Plano                   | 1                   | Camino normal                           |
| `A`         | Arena fina              | 2                   | Más lento, gasta más energía            |
| `D`         | Duna                    | 3                   | Aún más costoso                         |
| `O`         | Obstáculo               | —                   | No transitable                          |
| `Rover`     | Rover (posición actual) | —                   | Rover                                   |
| `B`         | Base de comunicación    | —                   | Punto donde puede transmitir y recargar |
| `PI`        | Punto de interés        | —                   | Donde puede usar el brazo robótico      |


In [2]:
import time 

import random


In [3]:
class Rover:
    def __init__(self, x, y, energia_max):
        self.x = x
        self.y = y
        self.energia = energia_max
        self.energia_max = energia_max
        self.muestras = 0
        self.comunicacion = True  # Solo True cuando está en base
        self.brazo_activo = False

    def mover(self, dx, dy, coste):
        if self.energia >= coste:
            self.x += dx
            self.y += dy
            self.energia -= coste
        else:
            print("Energía insuficiente para moverse.")

    def recargar(self):
        if self.energia<=self.energia_max:
            time.sleep(2)
        else:
            time.sleep(1)
        self.energia = self.energia_max
        print("Rover recargado en la base.")

    def recolectar(self):
        if self.brazo_activo and self.energia >= 2:
            self.muestras += 1
            self.energia -= 2
            print("Mineral recolectado.")
        else:
            print("Activa el brazo robótico o recarga energía.")

    def activar_brazo(self):
        self.brazo_activo = True
        print("Brazo robótico activado.")


In [12]:
class ContextoMarciano:
    def __init__(self, tamaño=15, energia_max=100,rover=[0,0],base=(0,0)):
        self.tamaño = tamaño
        self.mapa = self.generar_mapa()
        self.rover = Rover(rover[0], rover[1], energia_max)
        self.base = base
        self.puntos_interes = [(5, 5), (10, 8), (13, 13)]

    def generar_mapa(self):
        terrenos = ['.', 'A', 'D', 'O']
        pesos = [0.5, 0.25, 0.15, 0.1]
        return [[random.choices(terrenos, pesos)[0] for _ in range(15)] for _ in range(15)]

    def mostrar_mapa(self):
        for i in range(self.tamaño):
            fila = ""
            for j in range(self.tamaño):
                if (i, j) == (self.rover.x, self.rover.y):
                    fila += "Rover "
                elif (i, j) == self.base:
                    fila += "B "
                elif (i, j) in self.puntos_interes:
                    fila += "PI "
                else:
                    fila += self.mapa[i][j] + " "
            print(fila)
        print(f"Energía: {self.rover.energia} | Muestras: {self.rover.muestras}")
    def tipo_terreno(self, x, y):
        if 0 <= x < self.tamaño and 0 <= y < self.tamaño:
            return self.mapa[x][y]
        return 'O'

    def coste_movimiento(self, terreno):
        if terreno == '.': return 1
        if terreno == 'A': return 2
        if terreno == 'D': return 3
        return 999  # obstáculo o inválido

    def celda_valida(self, x, y):
        if 0 <= x < self.tamaño and 0 <= y < self.tamaño:
            return self.mapa[x][y] != 'O'
        return False
    def mover_aleatorio(self):
        # Posibles direcciones: arriba, abajo, izquierda, derecha
        direcciones = [(-1,0), (1,0), (0,-1), (0,1)]
        random.shuffle(direcciones)  # Para variar el orden

        for dx, dy in direcciones:
            nx, ny = self.rover.x + dx, self.rover.y + dy
            if self.celda_valida(nx, ny):
                terreno = self.tipo_terreno(nx, ny)
                coste = self.coste_movimiento(terreno)
                self.rover.mover(dx, dy, coste)
                return True  # movimiento exitoso
        return False  # no pudo moverse (bloqueado)
    def simular(self, pasos_max=100):
        print("🔴 Iniciando simulación del rover ARES...\n")
        self.rover.activar_brazo()

        for paso in range(pasos_max):
            if self.rover.energia <= 0:
                print("⚡ Energía agotada. Fin de la simulación.")
                break

            self.mostrar_mapa()

            # Si está sobre un punto de interés y tiene energía, recolecta
            if (self.rover.x, self.rover.y) in self.puntos_interes:
                self.rover.recolectar()
                # Elimina el punto de interés (ya recolectado)
                self.puntos_interes.remove((self.rover.x, self.rover.y))

            # Si está en la base y tiene muestras, recarga y “transmite”
            if (self.rover.x, self.rover.y) == self.base:
                self.rover.recargar()
                if self.rover.muestras > 0:
                    print(f"📡 Transmitiendo {self.rover.muestras} muestras a la base.")
                    self.rover.muestras = 0

            # Si ya visitó todos los POI, terminar
            if not self.puntos_interes:
                print("✅ Misión completa: todos los muestras recolectados y transmitidos.")
                break

            # Movimiento aleatorio
            if not self.mover_aleatorio():
                print("🚧 Rover bloqueado, no hay movimiento posible.")
                break

        print("🔚 Simulación finalizada.")


In [14]:
entorno = ContextoMarciano()
entorno.simular(pasos_max=5)

🔴 Iniciando simulación del rover ARES...

Brazo robótico activado.
Rover . A . . . . . . D D . . D D 
A A A A . . A D . O A D A D . 
. . . D . A . D A O A O A A . 
. A A D A D A . A . . A A . A 
A D O O . . A . A A O O . . O 
. . A . . PI A O A D D . . . . 
. . O D . . . . A D . D . . . 
A A . . O . D . . A D D . . . 
A O D A A A D A . D . A . . . 
A . A . . . A A . . D . . D . 
. A O . D . O D PI . A O . D . 
A . . O D . A O . A D A O . A 
. . A O . . . . O A A . D . . 
. . O A . . O . A . O . . PI A 
A A A A . . D . . . . D D . A 
Energía: 100 | Muestras: 0
Rover recargado en la base.
B Rover A . . . . . . D D . . D D 
A A A A . . A D . O A D A D . 
. . . D . A . D A O A O A A . 
. A A D A D A . A . . A A . A 
A D O O . . A . A A O O . . O 
. . A . . PI A O A D D . . . . 
. . O D . . . . A D . D . . . 
A A . . O . D . . A D D . . . 
A O D A A A D A . D . A . . . 
A . A . . . A A . . D . . D . 
. A O . D . O D PI . A O . D . 
A . . O D . A O . A D A O . A 
. . A O . . . . O A A . D . 